In [1]:
# Dependencies
import pandas as pd
import numpy as np

In [2]:
# Import files using pandas
items_df = pd.read_csv("generated_data\items_complete.csv")
players_df = pd.read_csv("generated_data\players_complete.csv")
purchase_df = pd.read_csv("generated_data\purchase_data_3.csv")

In [3]:
# Get the total number of players with .count(), using SN column b/c its short
player_count = players_df["SN"].count()

In [4]:
# Get the total count of unique items with len() since .unique() returns a list
unique_item_count = len(items_df["Item Name"].unique())

# Get the average purchase price by dividing the sum of the price column by total number of items
average_purchase_price = purchase_df["Price"].sum() / len(purchase_df["Price"])

# Get the total number of purchases
total_purchases = purchase_df["Price"].count()

# Get total revenue by adding up the revenue from the purchase_df
total_revenue = purchase_df["Price"].sum()


In [5]:
# Get % and # of male players
number_male = len(players_df.loc[players_df["Gender"] == "Male",:])

percent_male = (number_male / player_count)
percent_male = "{:.2%}".format(percent_male)

# Get % and # of female players
number_female = len(players_df.loc[players_df["Gender"] == "Female",:])

percent_female = (number_female / player_count)
percent_female = "{:.2%}".format(percent_female)

# Get % and # of other / non-disclosed players
number_other = len(players_df.loc[players_df["Gender"] == "Other / Non-Disclosed",:])

percent_other = (number_other / player_count)
percent_other = "{:.2%}".format(percent_other)

In [6]:
# Set up some dataframes etc to use for purchase analysis
male_df = purchase_df.loc[purchase_df["Gender"] == "Male",:]
female_df = purchase_df.loc[purchase_df["Gender"] == "Female",:]
other_df = purchase_df.loc[purchase_df["Gender"] == "Other / Non-Disclosed",:]

In [7]:
# Get purchase count broken by gender
male_purchase = len(male_df)
female_purchase = len(female_df)
other_purchase = len(other_df)

# Get total purchase value broken by gender
male_total_value = male_df["Price"].sum()
female_total_value = female_df["Price"].sum()
other_total_value = other_df["Price"].sum()

# Get average purchase price broken by gender
male_average_price = male_total_value / male_purchase
female_average_price = female_total_value / female_purchase
other_average_price = other_total_value / other_purchase

# Get normalized totals broken by gender, with code repurposed from stackoverflow
male_purchase_normalized = (male_df["Price"] - male_df["Price"].mean()) / (male_df["Price"].max() - male_df["Price"].min())
male_purchase_normalized_total = male_purchase_normalized.sum()

female_purchase_normalized = (female_df["Price"] - female_df["Price"].mean()) / (female_df["Price"].max() - female_df["Price"].min())
female_purchase_normalized_total = female_purchase_normalized.sum()

other_purchase_normalized = (other_df["Price"] - other_df["Price"].mean()) / (other_df["Price"].max() - other_df["Price"].min())
other_purchase_normalized_total = other_purchase_normalized.sum()

In [8]:
# Prepare to analyze age demographics
# first join purchase and players tables and clean it
players_purchase_joined_df = pd.merge(purchase_df, players_df, on="SN")
players_purchase_joined_df = players_purchase_joined_df.rename(columns={"Age_x": "Age",
                                                                       "Gender_x": "Gender"})
players_purchase_joined_clean_df = players_purchase_joined_df[["Purchase ID",
                                                             "SN",
                                                             "Age",
                                                             "Gender",
                                                             "Item ID",
                                                             "Item Name",
                                                             "Price",
                                                             "Player ID"]]
players_purchase_joined_clean_df.head(1)

# make bins and group names
bins = [0, 10, 14, 18, 22, 26, 30, 34, 38, 200]
bin_names = ["< 10", "10-14", "14-18", "18-22", "22-26", "26-30", "30-34", "34-38", "38+"]
players_purchase_joined_clean_df["Age Groups"] = pd.cut(players_purchase_joined_clean_df["Age"], bins, labels=bin_names)
players_purchase_joined_clean_df.head()

# Create a dataframe grouped by age groups
players_purchase_groups = players_purchase_joined_clean_df.groupby("Age Groups")

c:\users\t0nb3\Anaconda3\envs\PythonData\lib\site-packages\ipykernel\__main__.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [9]:
# Get purchase count by age groups
purchase_count_age_groups = players_purchase_groups["Purchase ID"].count()

# Get average purchase price by age groups
average_purchase_age_groups = players_purchase_groups["Price"].mean()

# Get total purchase value by age groups

total_purchase_age_groups = players_purchase_groups["Price"].sum()

# Get normalized totals by age groups
#age_purchase_normalized = (players_purchase_groups["Price"] - players_purchase_groups["Price"].mean()) / (players_purchase_groups["Price"].max() - players_purchase_groups["Price"].min())
#using more code taken from stackoverflow
age_purchase_normalized = players_purchase_joined_clean_df.groupby("Age Groups").transform(lambda x: (x - x.mean()) / x.std())
total_normalized_purchase_age_groups = age_purchase_normalized.sum()


In [40]:
# Prepare to analyze top spenders
# take cleaned table from above and group by SN to find top spenders SNs
players_purchase_sn_groups = players_purchase_joined_clean_df.groupby("SN")

sorted_top_spender_SN = pd.DataFrame(players_purchase_sn_groups["Price"].sum()).sort_values(["Price"], ascending=False)
top_five_spender_SN = sorted_top_spender_SN.head(5)

In [19]:
# get top 5 spender purchase count
SN_purchase_count = pd.DataFrame(players_purchase_sn_groups["SN"].count())
top_five_count = SN_purchase_count.loc[top_five_spender_SN.index.tolist()]
top_five_count = top_five_count.rename(columns={"SN":"Purchase Count"})

# get top 5 spender total purchase value
SN_purchase_price = pd.DataFrame(players_purchase_sn_groups["Price"].sum())
top_five_total = SN_purchase_price.loc[top_five_spender_SN.index.tolist()]

# get top 5 spender average purchase price
# have to first convert column to numeric data
top_five_counts = pd.to_numeric(top_five_count["Purchase Count"])
top_five_totals = pd.to_numeric(top_five_total["Price"])
top_five_average = top_five_totals / top_five_counts

# Spender Report
spender_report = top_five_spender_SN.rename(columns={"Price":"Total Price"})
spender_report["Purchase Count"] = top_five_count
spender_report["Average Purchase Price"] = top_five_average
spender_report["Total Purchase Value"] = top_five_total
spender_report = spender_report[["Purchase Count", "Average Purchase Price", "Total Purchase Value"]]
spender_report

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Jiskjask76,2,4.590,9.18
Sundaky74,2,3.705,7.41
Aidaira26,2,2.565,5.13
Eusty71,1,4.810,4.81
Chanirra64,1,4.780,4.78


In [48]:
# Prepare to analyze items and purchases
# first join the two tables and get rid of uninteresting
items_purchase_joined_df = pd.merge(purchase_df, items_df, on="Item ID")
items_purchase_joined_df = items_purchase_joined_df.rename(columns={"Item Name_x": "Item Name",
                                                                   "Price_x": "Price"})
items_purchase_joined_clean_df = items_purchase_joined_df[["Item ID",
                                                          "Purchase ID",
                                                          "Item Name",
                                                          "Price"]]

In [49]:
# ID 5 most popular items by purchase count
items_groups = items_purchase_joined_clean_df.groupby("Item ID")
sorted_popular_items = pd.DataFrame(items_groups.count())
sorted_popular_items = sorted_popular_items.rename(columns={"Item Name":"Times Purchased"})
sorted_popular_items = sorted_popular_items[["Times Purchased"]]
sorted_popular_items = sorted_popular_items.sort_values(["Times Purchased"], ascending=False)
top_five_popular_items = sorted_popular_items.head(5)
top_five_popular_items

,Times Purchased
Item ID,
94,3
90,2
111,2
64,2
154,2


In [112]:
# Get item name for top five item IDs
item_names = pd.DataFrame(items_groups["Item Name"].unique())
top_five_item_names = item_names.loc[top_five_popular_items.index.tolist()]

# Get item prices for top five item IDs
item_prices = pd.DataFrame(items_groups["Price"].unique())
top_five_item_prices = item_prices.loc[top_five_popular_items.index.tolist()]

# Get total purchase value for top five item IDs
item_value = pd.DataFrame(items_groups["Price"].sum())
top_five_item_values = item_value.loc[top_five_popular_items.index.tolist()]

,Price
Item ID,
94,10.92
90,8.24
111,3.58
64,4.84
154,8.22


In [22]:
purchase_df.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Iloni35,20,Male,93,Apocalyptic Battlescythe,4.49
1,1,Aidaira26,21,Male,12,Dawne,3.36
2,2,Irim47,17,Male,5,Putrid Fan,2.63
3,3,Irith83,17,Male,123,Twilight's Carver,2.55
4,4,Philodil43,22,Male,154,Feral Katana,4.11


In [15]:
items_df.head(1)

,Item ID,Item Name,Price
0,0,Splinter,1.89


In [23]:
players_df.head(10)

,Player ID,SN,Age,Gender
0,0,Marughi89,21,Male
1,1,Lirtedy26,40,Male
2,2,Chamistast30,7,Male
3,3,Lisirra25,24,Male
4,4,Lirtim36,23,Male
5,5,Undimsya85,17,Female
6,6,Lirtassa77,20,Female
7,7,Mindirra92,23,Male
8,8,Undirrasta89,23,Male
9,9,Iskjaskst81,15,Male


In [18]:
# Things to do
#PLAYER COUNT
# - Total Number of Players player_count
#PURCHASING ANALYSIS(TOTAL)
# - Number of Unique Items unique_item_count
# - Average Purchase Price average_purchase_price
# - Total Number of Purchases total_purchases
# - Total Revenue total_revenue
#GENDER DEMOGRAPHICS
# - Percentage and Count of Male Players percent_male, number_male
# - Percentage and Count of Female Players percent_female, number_female
# - Percentage and Count of Other / Non-Disclosed percent_other, number_other
#PURCHASING ANALYSIS (GENDER)
#>broken by gender
# - Purchase Count male_purchase, female_purchase, other_purchase
# - Average Purchase Price male_average_price, female_average_price, other_average_price
# - Total Purchase Value male_total_value, female_total_value, other_total_value
# - Normalized Totals male_purchase_normalized_total, female_purchase_normalized_total, other_purchase_normalized_total
#AGE DEMOGRAPHICS
#>broken into bins of 4 years (ie <10, 10-14, 15-19,...)
# - Purchase Count purchase_count_age_groups
# - Average Purchase Price average_purchase_age_groups
# - Total Purchase Value total_purchase_age_groups
# - Normalized Totals total_normalized_purchase_age_groups
#TOP SPENDERS
#>id top 5 spenders then list (in a table):
# - SN top_five_spender_SN
# - Purchase Count top_five_count
# - Average Purchase Price top_five_average
# - Total Purchase Value top_five_total
#MOST POPULAR ITEMS
#>id top 5 items by purchase count then list (in a table): 
# - Item ID index of all tables
# - Item Name top_five_item_names
# - Purchase Count top_five_popular_items
# - Item Price top_five_item_prices
# - Total Purchase Value top_five_item_values
#MOST PROFITABLE ITEMS

#items table: items_purchase_joined_clean_df

#example report table all put together from TOP SPENDERS section